In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import geopandas as gpd
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


/kaggle/input/geospatial-learn-course-data/CA_county_median_age.csv
/kaggle/input/geospatial-learn-course-data/top_universities.csv
/kaggle/input/geospatial-learn-course-data/japan-prefecture-population.csv
/kaggle/input/geospatial-learn-course-data/CA_county_population.csv
/kaggle/input/geospatial-learn-course-data/earthquakes1970-2014.csv
/kaggle/input/geospatial-learn-course-data/Philippines_AL258.kml
/kaggle/input/geospatial-learn-course-data/starbucks_locations.csv
/kaggle/input/geospatial-learn-course-data/purple_martin.csv
/kaggle/input/geospatial-learn-course-data/CA_county_high_earners.csv
/kaggle/input/geospatial-learn-course-data/PhillyHealth_Air_Monitoring_Stations/PhillyHealth_Air_Monitoring_Stations/PhillyHealth_Air_Monitoring_Stations.cpg
/kaggle/input/geospatial-learn-course-data/PhillyHealth_Air_Monitoring_Stations/PhillyHealth_Air_Monitoring_Stations/PhillyHealth_Air_Monitoring_Stations.shx
/kaggle/input/geospatial-learn-course-data/PhillyHealth_Air_Monitoring_Station

# how hospitals have been responding to crash collisions in New York City.

In [2]:
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon

import folium
from folium import Choropleth, Marker
from folium.plugins import HeatMap, MarkerCluster

In [3]:
# To Visualize
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='1000px')

In [4]:
collisions = gpd.read_file("../input/geospatial-learn-course-data/NYPD_Motor_Vehicle_Collisions/NYPD_Motor_Vehicle_Collisions/NYPD_Motor_Vehicle_Collisions.shp")
collisions.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET,CROSS STRE,OFF STREET,...,CONTRIBU_2,CONTRIBU_3,CONTRIBU_4,UNIQUE KEY,VEHICLE TY,VEHICLE _1,VEHICLE _2,VEHICLE _3,VEHICLE _4,geometry
0,07/30/2019,0:00,BRONX,10464,40.841100,-73.784960,"(40.8411, -73.78496)",None,None,121 PILOT STREET,...,Unspecified,None,None,4180045,Sedan,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,None,None,POINT (1043750.211 245785.815)
1,07/30/2019,0:10,QUEENS,11423,40.710827,-73.770660,"(40.710827, -73.77066)",JAMAICA AVENUE,188 STREET,None,...,None,None,None,4180007,Sedan,Sedan,None,None,None,POINT (1047831.185 198333.171)
2,07/30/2019,0:25,None,None,40.880318,-73.841286,"(40.880318, -73.841286)",BOSTON ROAD,None,None,...,None,None,None,4179575,Sedan,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (1028139.293 260041.178)
3,07/30/2019,0:35,MANHATTAN,10036,40.756744,-73.984590,"(40.756744, -73.98459)",None,None,155 WEST 44 STREET,...,None,None,None,4179544,Box Truck,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (988519.261 214979.320)
4,07/30/2019,10:00,BROOKLYN,11223,40.600090,-73.965910,"(40.60009, -73.96591)",AVENUE T,OCEAN PARKWAY,None,...,None,None,None,4180660,Station Wagon/Sport Utility Vehicle,Bike,None,None,None,POINT (993716.669 157907.212)


In [5]:
collisions.columns

Index(['DATE', 'TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE',
       'LOCATION', 'ON STREET', 'CROSS STRE', 'OFF STREET', 'NUMBER OF',
       'NUMBER O_1', 'NUMBER O_2', 'NUMBER O_3', 'NUMBER O_4', 'NUMBER O_5',
       'NUMBER O_6', 'NUMBER O_7', 'CONTRIBUTI', 'CONTRIBU_1', 'CONTRIBU_2',
       'CONTRIBU_3', 'CONTRIBU_4', 'UNIQUE KEY', 'VEHICLE TY', 'VEHICLE _1',
       'VEHICLE _2', 'VEHICLE _3', 'VEHICLE _4', 'geometry'],
      dtype='object')

In [6]:
collisions.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 261905 entries, 0 to 261904
Data columns (total 30 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   DATE        261905 non-null  object  
 1   TIME        261905 non-null  object  
 2   BOROUGH     199243 non-null  object  
 3   ZIP CODE    199200 non-null  object  
 4   LATITUDE    261905 non-null  float64 
 5   LONGITUDE   261905 non-null  float64 
 6   LOCATION    261905 non-null  object  
 7   ON STREET   225789 non-null  object  
 8   CROSS STRE  185029 non-null  object  
 9   OFF STREET  23998 non-null   object  
 10  NUMBER OF   261905 non-null  float64 
 11  NUMBER O_1  261892 non-null  float64 
 12  NUMBER O_2  261905 non-null  int64   
 13  NUMBER O_3  261905 non-null  int64   
 14  NUMBER O_4  261905 non-null  int64   
 15  NUMBER O_5  261905 non-null  int64   
 16  NUMBER O_6  261905 non-null  int64   
 17  NUMBER O_7  261905 non-null  int64   
 18  CONTRIBUTI  2601

In [7]:
collision = collisions[['DATE','TIME','LATITUDE','LONGITUDE','LOCATION','UNIQUE KEY','geometry']]
collision.head(10)

,DATE,TIME,LATITUDE,LONGITUDE,LOCATION,UNIQUE KEY,geometry
0,07/30/2019,0:00,40.841100,-73.784960,"(40.8411, -73.78496)",4180045,POINT (1043750.211 245785.815)
1,07/30/2019,0:10,40.710827,-73.770660,"(40.710827, -73.77066)",4180007,POINT (1047831.185 198333.171)
2,07/30/2019,0:25,40.880318,-73.841286,"(40.880318, -73.841286)",4179575,POINT (1028139.293 260041.178)
3,07/30/2019,0:35,40.756744,-73.984590,"(40.756744, -73.98459)",4179544,POINT (988519.261 214979.320)
4,07/30/2019,10:00,40.600090,-73.965910,"(40.60009, -73.96591)",4180660,POINT (993716.669 157907.212)
5,07/30/2019,10:50,40.721060,-73.759450,"(40.72106, -73.75945)",4179812,POINT (1050928.749 202069.687)
6,07/30/2019,10:55,40.676228,-73.761120,"(40.676228, -73.76112)",4180464,POINT (1050510.380 185734.852)
7,07/30/2019,11:00,40.752975,-73.910240,"(40.752975, -73.91024)",4180993,POINT (1009118.946 213618.517)
8,07/30/2019,11:03,40.630337,-73.993420,"(40.630337, -73.99342)",4179981,POINT (986076.414 168925.212)
9,07/30/2019,11:40,40.757760,-73.783400,"(40.75776, -73.7834)",4179801,POINT (1044256.932 215423.295)


In [8]:
collision.isnull().sum()

DATE          0
TIME          0
LATITUDE      0
LONGITUDE     0
LOCATION      0
UNIQUE KEY    0
geometry      0
dtype: int64

In [9]:
m_1 = folium.Map(location=[40.7, -74], zoom_start=11) 

# Visualize the collision data
HeatMap(data=collision[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m_1)
embed_map(m_1, "collision.html")

In [10]:
hospitals = gpd.read_file("../input/geospatial-learn-course-data/nyu_2451_34494/nyu_2451_34494/nyu_2451_34494.shp")
hospitals.head()

,id,name,address,zip,factype,facname,capacity,capname,bcode,xcoord,ycoord,latitude,longitude,geometry
0,317000001H1178,BRONX-LEBANON HOSPITAL CENTER - CONCOURSE DIVI...,1650 Grand Concourse,10457,3102,Hospital,415,Beds,36005,1008872.0,246596.0,40.843490,-73.911010,POINT (1008872.000 246596.000)
1,317000001H1164,BRONX-LEBANON HOSPITAL CENTER - FULTON DIVISION,1276 Fulton Ave,10456,3102,Hospital,164,Beds,36005,1011044.0,242204.0,40.831429,-73.903178,POINT (1011044.000 242204.000)
2,317000011H1175,CALVARY HOSPITAL INC,1740-70 Eastchester Rd,10461,3102,Hospital,225,Beds,36005,1027505.0,248287.0,40.848060,-73.843656,POINT (1027505.000 248287.000)
3,317000002H1165,JACOBI MEDICAL CENTER,1400 Pelham Pkwy,10461,3102,Hospital,457,Beds,36005,1027042.0,251065.0,40.855687,-73.845311,POINT (1027042.000 251065.000)
4,317000008H1172,LINCOLN MEDICAL & MENTAL HEALTH CENTER,234 E 149 St,10451,3102,Hospital,362,Beds,36005,1005154.0,236853.0,40.816758,-73.924478,POINT (1005154.000 236853.000)


In [11]:
hospitals.shape

(62, 14)

In [12]:
hospitals.nunique()

id           62
name         62
address      62
zip          46
factype       1
facname       1
capacity     57
capname       2
bcode         5
xcoord       62
ycoord       62
latitude     62
longitude    62
geometry     62
dtype: int64

In [13]:
m_2 = folium.Map(location=[40.7, -74], zoom_start=11) 

#  Visualize the hospital locations
for idx, row in hospitals.iterrows():
    Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(m_2)

        
# Show the map
embed_map(m_2, "q_2.html")

In [14]:
coverage = gpd.GeoDataFrame(geometry=hospitals.geometry).buffer(10000)
my_union = coverage.geometry.unary_union
outside_range = collision.loc[~collision["geometry"].apply(lambda x: my_union.contains(x))]


In [15]:
outside_range.shape

(39595, 7)

In [16]:
percentage = round(100*len(outside_range)/len(collisions), 2)
print("Percentage of collisions more than 10 km away from the closest hospital: {}%".format(percentage))

Percentage of collisions more than 10 km away from the closest hospital: 15.12%


In [17]:
# Make a recomender
def best_hospital(collision_location):
    idx_min = hospitals.geometry.distance(collision_location).idxmin()
    my_hospital = hospitals.iloc[idx_min]
    name = my_hospital["name"]
    return name

In [18]:
highest_demand = outside_range.geometry.apply(best_hospital).value_counts().idxmax()

In [19]:
highest_demand

'JAMAICA HOSPITAL MEDICAL CENTER'

In [20]:
m_6 = folium.Map(location=[40.7, -74], zoom_start=10) 

coverage = gpd.GeoDataFrame(geometry=hospitals.geometry).buffer(10000)
folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m_6)
HeatMap(data=outside_range[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m_6)
folium.LatLngPopup().add_to(m_6)

embed_map(m_6, 'm_6.html')